In [16]:
import pandas as pd
import requests
import json
from bs4 import BeautifulSoup
# Helper function to find data between two strings
def find_between( s, first, last ):
    try:
        start = s.index( first ) + len( first )
        end = s.index( last, start )
        return s[start:end]
    except ValueError:
        return ""

# Helper function to check if a string is a float
def isfloat(num):
    try:
        float(num)
        return True
    except ValueError:
        return False

In [17]:
pid=str(1410020101)
pickMembers1=str(1.15)
pickMembers2=str(2.1)
startMonth="01"
startYear=str(2022)
endMonth=str(10)
endYear=str(2022)
referencePeriods="20220601%2C20221001"

url = 'https://www150.statcan.gc.ca/t1/tbl1/en/tv.action?pid=' + pid + '&pickMembers%5B0%5D='+ pickMembers1 + '&pickMembers%5B1%5D='+ pickMembers2 + '&cubeTimeFrame.startMonth='+ startMonth + '&cubeTimeFrame.startYear=' + startYear + '&cubeTimeFrame.endMonth=10&cubeTimeFrame.endYear=2022&referencePeriods=' + referencePeriods
page = requests.get(url)
soup = BeautifulSoup(page.content, 'html.parser')
result = find_between(soup.prettify(), 'tableContainerElement = $(".tableContainer").clone();', 'window.addEventListener("resize", function() {') + 'end'
data = find_between(result, 'prepareTable(', '\n')[:-2]
json_data = json.loads(data)
rows = json_data['rows']

In [18]:
new_rows = []
for row in rows:
    values = row['values']
    for value in values:
        new_rows.append(value["value"])

In [23]:
category_names = ["Geography", "Type of employee"]
category_dict = {}
for name in category_names:
    # print((item for item in json_data['headers']["columnHeaders"] if item["name"] == name)["values"])
    category = next(item for item in json_data['headers']["columnHeaders"] if item["name"] == name)
    category_dict[name] = category["values"][0]["value"]

<>:4: SyntaxWarning: 'generator' object is not subscriptable; perhaps you missed a comma?
<>:4: SyntaxWarning: 'generator' object is not subscriptable; perhaps you missed a comma?
/var/folders/0p/c1842hdx3cl1f16fhby8ffhr0000gn/T/ipykernel_50510/303712722.py:4: SyntaxWarning: 'generator' object is not subscriptable; perhaps you missed a comma?
  print((item for item in json_data['headers']["columnHeaders"] if item["name"] == name)["values"])
/var/folders/0p/c1842hdx3cl1f16fhby8ffhr0000gn/T/ipykernel_50510/303712722.py:4: SyntaxWarning: 'generator' object is not subscriptable; perhaps you missed a comma?
  print((item for item in json_data['headers']["columnHeaders"] if item["name"] == name)["values"])
/var/folders/0p/c1842hdx3cl1f16fhby8ffhr0000gn/T/ipykernel_50510/303712722.py:4: SyntaxWarning: 'generator' object is not subscriptable; perhaps you missed a comma?
  print((item for item in json_data['headers']["columnHeaders"] if item["name"] == name)["values"])
/var/folders/0p/c1842hdx3

TypeError: 'generator' object is not subscriptable

In [12]:
category_names = ["Geography", "Type of employee"]
category_dict = {}
for name in category_names:
    category = next(item for item in json_data['headers']["columnHeaders"] if item["name"] == name)
    category_dict[name] = category["values"][0]["value"]




In [20]:
# Return the headers for the data table.
headers = next(item for item in json_data['headers']["columnHeaders"] if item["name"] == "Reference period")
header_values = []
for item in headers["values"]:
        header_values.append(item["value"])
rows_values = {}
key = ""
data = []
index = 0
for row in new_rows:
    temp_data = {}
    if not isfloat(row):
        key = row
        rows_values[key] = []
        data = []
        index = 0
    if isfloat(row):
        data.append(float(row))
        rows_values[key] = data
        index += 1

In [21]:
test_data = []
keys = []
for key, value in rows_values.items():
    index=0
    temp_data = {}
    temp_data["key"] = key
    key = key.replace(" ", "_").replace(",","").replace("(","").replace(")","").replace("-","_").replace("__","_").lower()
    keys.append(key)
    for i in value:
        temp_data[header_values[index]] = i
        index += 1
    test_data.append(temp_data)

df = pd.DataFrame(test_data).transpose().drop("key")
df.columns = keys
df["date"] = soup.find_all('meta', attrs={'name': 'dcterms.issued'})[0]['content']

for key, value in category_dict.items():
    df[key] = value
print(df.head())

               industrial_aggregate_including_unclassified_businesses  \
June 2022                                                 20406.0       
July 2022                                                 20031.0       
August 2022                                               20409.0       
September 2022                                            20627.0       
October 2022                                              20915.0       

               industrial_aggregate_excluding_unclassified_businesses  \
June 2022                                                 20243.0       
July 2022                                                 19886.0       
August 2022                                               20268.0       
September 2022                                            20485.0       
October 2022                                              20730.0       

               goods_producing_industries forestry_logging_and_support  \
June 2022                          4803.0       